<a href="https://colab.research.google.com/github/mrartis1/MA-MachineLearning/blob/main/DNN1Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DNN1 Testing

In [5]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [8]:
traindata = pd.read_csv('/content/drive/MyDrive/Colab Results/Data/Training.csv', header=None)
testdata = pd.read_csv('/content/drive/MyDrive/Colab Results/Data/Testing.csv', header=None)

In [9]:
X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,0]
C = testdata.iloc[:,0]
T = testdata.iloc[:,1:42]

trainX = np.array(X)
testT = np.array(T)

trainX.astype(float)
testT.astype(float)

scaler = Normalizer().fit(trainX)
trainX = scaler.transform(trainX)

scaler = Normalizer().fit(testT)
testT = scaler.transform(testT)

y_train = np.array(Y)
y_test = np.array(C)


X_train = np.array(trainX)
X_test = np.array(testT)

batch_size = 64

In [10]:
# 1. define the network
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn1layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn1layer/dnn1layer_model.hdf5")

Epoch 1/100
7720/7720 [==============================] - 42s 5ms/step - loss: 0.0209 - accuracy: 0.9927

Epoch 00001: loss improved from inf to 0.02090, saving model to kddresults/dnn1layer/checkpoint-01.hdf5
Epoch 2/100
7720/7720 [==============================] - 39s 5ms/step - loss: 0.0097 - accuracy: 0.9971

Epoch 00002: loss improved from 0.02090 to 0.00967, saving model to kddresults/dnn1layer/checkpoint-02.hdf5
Epoch 3/100
7720/7720 [==============================] - 38s 5ms/step - loss: 0.0080 - accuracy: 0.9975

Epoch 00003: loss improved from 0.00967 to 0.00803, saving model to kddresults/dnn1layer/checkpoint-03.hdf5
Epoch 4/100
7720/7720 [==============================] - 38s 5ms/step - loss: 0.0070 - accuracy: 0.9978

Epoch 00004: loss improved from 0.00803 to 0.00699, saving model to kddresults/dnn1layer/checkpoint-04.hdf5
Epoch 5/100
7720/7720 [==============================] - 38s 5ms/step - loss: 0.0063 - accuracy: 0.9980

Epoch 00005: loss improved from 0.00699 to 0.00

In [12]:
score = []
name = []
from sklearn.metrics import confusion_matrix
import os
for file in os.listdir("/content/drive/MyDrive/Colab Results/kddresults/dnn1layer/"):
  if file.endswith(".hdf5"):
    model.load_weights("/content/drive/MyDrive/Colab Results/kddresults/dnn1layer/"+file)
    y_train1 = y_test
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_train1, y_pred)
    recall = recall_score(y_train1, y_pred , average="binary")
    precision = precision_score(y_train1, y_pred , average="binary")
    f1 = f1_score(y_train1, y_pred, average="binary")
    print("----------------------------------------------")
    print("accuracy")
    print("%.3f" %accuracy)
    print("recall")
    print("%.3f" %recall)
    print("precision")
    print("%.3f" %precision)
    print("f1score")
    print("%.3f" %f1)
    score.append(accuracy)
    name.append(file)

----------------------------------------------
accuracy
0.922
recall
0.908
precision
0.994
f1score
0.949
----------------------------------------------
accuracy
0.922
recall
0.907
precision
0.996
f1score
0.949
----------------------------------------------
accuracy
0.923
recall
0.909
precision
0.995
f1score
0.950
----------------------------------------------
accuracy
0.921
recall
0.906
precision
0.996
f1score
0.949
----------------------------------------------
accuracy
0.923
recall
0.908
precision
0.995
f1score
0.950
----------------------------------------------
accuracy
0.922
recall
0.906
precision
0.996
f1score
0.949
----------------------------------------------
accuracy
0.923
recall
0.909
precision
0.996
f1score
0.950
----------------------------------------------
accuracy
0.923
recall
0.908
precision
0.996
f1score
0.950
----------------------------------------------
accuracy
0.923
recall
0.908
precision
0.996
f1score
0.950
----------------------------------------------
accuracy

In [13]:
model.load_weights("/content/drive/MyDrive/Colab Results/kddresults/dnn1layer/"+name[score.index(max(score))])
pred = (model.predict(X_test) > 0.5).astype("int32")
#proba = model.predict_proba(X_test)
np.savetxt("/content/drive/MyDrive/Colab Results/dnnres/dnn1predicted.txt", pred)
#np.savetxt("/content/drive/MyDrive/Colab Results/dnnres/dnn1probability.txt", proba)

accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred , average="binary")
precision = precision_score(y_test, pred , average="binary")
f1 = f1_score(y_test, pred, average="binary")

print("----------------------------------------------")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)

----------------------------------------------
accuracy
0.930
precision
0.998
racall
0.914
f1score
0.954


In [14]:
model.load_weights("/content/drive/MyDrive/Colab Results/kddresults/dnn1layer/"+name[score.index(max(score))])
pred = (model.predict(X_test) > 0.5).astype("int32")
#proba = model.predict_proba(X_test)
np.savetxt("/content/drive/MyDrive/Colab Results/dnnres/dnn1predicted.txt", pred)
#np.savetxt("/content/drive/MyDrive/Colab Results/dnnres/dnn1probability.txt", proba)
accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred , average="binary")
precision = precision_score(y_test, pred , average="binary")
f1 = f1_score(y_test, pred, average="binary")
print("----------------------------------------------")
print("accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("racall")
print("%.3f" %recall)
print("f1score")
print("%.3f" %f1)
print(model.summary())

----------------------------------------------
accuracy
0.930
precision
0.998
racall
0.914
f1score
0.954
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              43008     
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 44,033
Trainable params: 44,033
Non-trainable params: 0
_________________________________________________________________
None
